### 心理认知偏差
对交易有影响的交易偏差  
- 损失厌恶：对避免损失有一种强烈的偏好，不赔钱远比赚钱重要。
- 沉没成本效应：更重视已经花掉的钱，而不是未来可能要花的钱。
- 处置效应：早早兑现利润，却仍损失持续下去。
- 结果偏好：只会根据一个决策的结果来判断它的好坏，而不去考虑决策本身的质量。
- 近期偏好：更重视近期的数据或经验，忽视早期的数据或经验。
- 锚定效应：过于依赖容易获得的信息。
- 潮流效应：盲目相信一件事，只因为其他许多人都相信他。
- 信奉小数法则：从太少的信息中得出没有一句的结论。

## 海龟系统简介
海龟交易系统是一个完整的交易系统,它有一个完整的交易系统所应该有的所有成分，涵盖了成功交易中的每一个必要决策：
- 市场：买卖什么？
- 头寸规模：买卖多少？
- 入市：什么时候买卖？
- 止损：什么时候放弃一个亏损的头寸？
- 退出：什么时候退出一个盈利的头寸？
- 战术：怎么买卖？

“海龟们从不去预测市场的动向，而是会寻找市场处于某种特定状态的指示信号。优秀的交易者不会试着预测市场下一步会怎么样；相反，他们会观察指示信号，判断市场现在正处于什么样的状态中。”

### 趋势捕捉

唐奇安通道：当价格冲破上轨是可能的买入信号，反之，冲破下轨时是可能的卖出信号。  
上线=MAX(最高价,n)  
下线=MIN(最低价,n)  
中线=(上线+下线)/2

海龟交易的具体规则是：  

- 当今天的收盘价，大于过去20个交易日中的最高价时，以收盘价买入；  
- 买入后，当收盘价小于过去10个交易日中的最低价时，以收盘价卖出。

海龟交易系统本质上是一个趋势跟随的系统，但是最值得我们学习的，是资金管理尤其是分批建仓及动态止损的部分。

N值：仓位管理的核心，涉及加仓及止损。N值与技术指标平均真实波幅 ATR很相似。  
真实波幅： 真实波幅是以下三个值中的最大值  
1、当前交易日最高价和最低价的波幅 2、前一交易日的收盘价与当前交易日最高价的波幅 3、前一交易日的收盘价与当前交易日最低价的波幅
用公式写就是：TrueRange=Max(High−Low,abs(High−PreClose),abs(PreClose−Low))

N值计算公式为：

N=（PreN[−19：]+TrueRange）/20
其中 preN为前面N值，TrueRange为当前的真实波幅,此公式的真是含义为计算之前20天（包括今天在内）的N的平均值。  
有些海龟交易系统用的是ATR来代替N值，ATR为真实波幅的20日平均

### 买卖单位及首次建仓
Unit=（1%∗Account）/N  
首次建仓的时候，当捕捉到趋势，即价格突破唐奇安上轨时，买入1个unit。
其意义就是，让一个N值的波动与你总资金1%的波动对应，如果买入1unit单位的资产，当天震幅使得总资产的变化不超过1%。  
例如：
现在你有10万元资金，1%波动就是1000元。假如标X的N值为0.2元，1000元÷0.2元=5000股。也就是说，你的第一笔仓位应该是在其突破上轨（假设为5元）时立刻买入5000股，耗资25000元。

### 动态止损
当价格比最后一次买入价格下跌2N时，则卖出全部头寸止损。
接上面的例子，最后一次加仓价格为5.2。假如此时N值0.2元。 当价格下跌到$ 5.2 - 2*0.2 = 4.8$元时，清仓。 持仓成本为 （5+5.1+5.2）*5000/15000 = 5.1元。 此时亏损 （5.1-4.8）*15000 = 4500元 对于10万来说 这波亏损4.5%  

### 止盈

当股价跌破10日唐奇安通道下沿，清空头寸结束本次交易。

原始的海龟交易采用唐奇安通道来捕捉趋势，虽然能捕捉到大趋势，但是在震荡的情况下表现不如人意，不过这也是所有趋势型策略的通病。但是回测小的特点可以应用在资金管理上，择时可以看看多因子、动量反转等。

## 海龟交易规则Python实现代码

In [14]:
# In order to get Apple stock price data from yahoo
import yfinance as yf

# Basic data manipulation libraries
import datetime
import numpy as np
import pandas as pd
import math
import talib

# Data viz library
import matplotlib.pyplot as plt

# We download the stock price from start date until end date
df = yf.download("0857.HK", start="2022-01-01", end="2023-10-27")

df.tail()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-10-19,5.70,5.73,5.60,5.64,5.64,99122480
2023-10-20,5.63,5.69,5.60,5.62,5.62,76363436
2023-10-24,5.54,5.56,5.44,5.53,5.53,96718947
2023-10-25,5.55,5.56,5.24,5.31,5.31,192167444
2023-10-26,5.31,5.35,5.24,5.32,5.32,93655106


In [15]:
#hs=get_daily_data('沪深300','20180101','')[['close','open','high','low','vol']]
#最近N1个交易日最高价
df['up']=talib.MAX(df.High,timeperiod=20).shift(1)
#最近N2个交易日最低价
df['down']=talib.MIN(df.Low,timeperiod=10).shift(1)
#每日真实波动幅度
df['ATR']=talib.ATR(df.High,df.Low,df.Close,timeperiod=20)
df.tail()

,Open,High,Low,Close,Adj Close,Volume,up,down,ATR
Date,,,,,,,,,
2023-10-19,5.70,5.73,5.60,5.64,5.64,99122480,6.02,5.35,0.162156
2023-10-20,5.63,5.69,5.60,5.62,5.62,76363436,6.02,5.45,0.158548
2023-10-24,5.54,5.56,5.44,5.53,5.53,96718947,6.02,5.51,0.159621
2023-10-25,5.55,5.56,5.24,5.31,5.31,192167444,6.02,5.44,0.167640
2023-10-26,5.31,5.35,5.24,5.32,5.32,93655106,6.02,5.24,0.164758


In [16]:
#pip install pyecharts==0.5.11
def my_strategy(data):
    x1=data.Close>data.up
    x2=data.Close.shift(1)<data.up.shift(1)
    x=x1&x2
    y1=data.Close<data.down
    y2=data.Close.shift(1)>data.down.shift(1)
    y=y1&y2
    data.loc[x,'signal']='buy'
    data.loc[y,'signal']='sell'
    buy_date=(data[data.signal=='buy'].index).strftime('%Y%m%d')
    sell_date=(data[data.signal=='sell'].index).strftime('%Y%m%d')
    buy_close=data[data.signal=='buy'].Close.round(2).tolist()
    sell_close=data[data.signal=='sell'].Close.round(2).tolist()
    return (buy_date,buy_close,sell_date,sell_close)

In [17]:
#对K线图和唐奇安通道进行可视化
#from pyecharts import line, kline, grid, bar

from pyecharts import Overlap
from pyecharts import Kline
from pyecharts import Line
from pyecharts import Bar
from pyecharts import Grid
from pyecharts import EffectScatter

grid = Grid()
attr=[str(t) for t in df.index.strftime('%Y%m%d')]

v1=np.array(df.loc[:,['Open','Close','Low','High']])
v2=np.array(df.up)
v3=np.array(df.down)

kline = Kline("唐奇安通道",title_text_size=15)
kline.add("K线图", attr, v1.round(1),is_datazoom_show=True,)



In [18]:
# 成交量
bar = Bar()
bar.add("成交量", attr, df['Volume'],tooltip_tragger="axis", is_legend_show=False, 
        is_yaxis_show=False, yaxis_max=5*max(df["Volume"]))




In [19]:
line = Line()
line.add("上轨线", attr, v2.round(1),is_datazoom_show=True,
         is_smooth=True,is_symbol_show=False,line_width=1.5)
line.add("下轨线", attr, v3.round(1),is_datazoom_show=True,
         is_smooth=True,is_symbol_show=False,line_width=1.5)


In [20]:
#添加买卖信号
bd,bc,sd,sc=my_strategy(df)
es = EffectScatter("buy")
es.add( "sell", sd, sc, )
es.add("buy", bd, bc,symbol="triangle",)
    
overlap = Overlap(width=2000, height=600)
overlap.add(kline)
overlap.add(line)
overlap.add(bar,yaxis_index=1, is_add_yaxis=True)
overlap.add(es)

grid.add(overlap, grid_right="10%")
grid